In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 10.1 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 11.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 11.5 MB

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
# from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

In [7]:
import os
import tempfile
# from git import Repo

# def clone_repository(repo_url):
#     """Clones a GitHub repository to a temporary directory.

#     Args:
#         repo_url: The URL of the GitHub repository.

#     Returns:
#         The path to the cloned repository.
#     """
#     repo_name = repo_url.split("/")[-1]  # Extract repository name from URL
#     repo_path = f"/content/{repo_name}"
#     Repo.clone_from(repo_url, str(repo_path))
#     return str(repo_path)

def clone_repository(repo_url: str) -> str:
    """Clone a repository and return its path"""
    # Use tempfile to get a temporary directory that's writable
    temp_dir = tempfile.mkdtemp()
    repo_name = repo_url.split("/")[-1]  # Extract repository name from URL
    repo_path = os.path.join(temp_dir, repo_name)
    
    # Clone the repository
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)

In [8]:
path = clone_repository("https://github.com/CoderAgent/SecureAgent")

In [9]:
print(path)

/var/folders/pg/wwfxmq4x1vx_t00m5pz1z4nr0000gp/T/tmpwh8fbp6o/SecureAgent


In [10]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [11]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Get relative path from repo root
        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
        for root, _, files in os.walk(repo_path):
            # Skip if current directory is in ignored directories
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            # Process each file in current directory
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

In [12]:
file_content = get_main_files_content(path)

In [13]:
file_content

[{'name': 'src/app.ts',
  'content': 'import { Octokit } from "@octokit/rest";\nimport { createNodeMiddleware } from "@octokit/webhooks";\nimport { WebhookEventMap } from "@octokit/webhooks-definitions/schema";\nimport * as http from "http";\nimport { App } from "octokit";\nimport { Review } from "./constants";\nimport { env } from "./env";\nimport { processPullRequest } from "./review-agent";\nimport { applyReview } from "./reviews";\n\n// This creates a new instance of the Octokit App class.\nconst reviewApp = new App({\n  appId: env.GITHUB_APP_ID,\n  privateKey: env.GITHUB_PRIVATE_KEY,\n  webhooks: {\n    secret: env.GITHUB_WEBHOOK_SECRET,\n  },\n});\n\nconst getChangesPerFile = async (payload: WebhookEventMap["pull_request"]) => {\n  try {\n    const octokit = await reviewApp.getInstallationOctokit(\n      payload.installation.id\n    );\n    const { data: files } = await octokit.rest.pulls.listFiles({\n      owner: payload.repository.owner.login,\n      repo: payload.repository.na

In [14]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [44]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

In [45]:
embeddings

array([ 1.81737561e-02, -3.02658021e-03, -4.77465168e-02,  1.86379459e-02,
        3.14538553e-02,  1.87256038e-02, -1.52535010e-02, -6.77293763e-02,
       -1.26903830e-02,  1.28427902e-02,  5.80700971e-02,  4.00234573e-02,
        3.27073075e-02,  7.12998137e-02,  5.56373149e-02,  1.68628637e-02,
        6.97603971e-02, -5.02620190e-02,  6.13149907e-03, -1.46559495e-02,
       -4.51954640e-03,  4.82934080e-02, -2.53051538e-02, -1.97862601e-03,
       -4.36902978e-02, -2.41507683e-02,  1.29505266e-02, -3.78607749e-03,
       -2.05718856e-02,  1.09819151e-01,  3.07671353e-03, -2.80443449e-02,
       -1.55807203e-02, -1.24790575e-02,  1.75239234e-06, -2.93753087e-03,
       -1.43048689e-02,  4.88385893e-02, -6.21115342e-02,  2.95061544e-02,
       -1.40470751e-02,  2.20708121e-02,  1.13068130e-02,  4.70892638e-02,
        7.58303842e-03, -8.30338395e-05,  6.67821839e-02, -1.21320179e-02,
        4.39393567e-03,  2.47454196e-02,  1.02529237e-02, -6.54435391e-03,
       -5.53144375e-03, -

In [25]:
from dotenv import load_dotenv
load_dotenv()

import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pinecone_index = pc.Index("codebase-rag")


In [26]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

/var/folders/pg/wwfxmq4x1vx_t00m5pz1z4nr0000gp/T/ipykernel_89637/2416920381.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
/var/folders/pg/wwfxmq4x1vx_t00m5pz1z4nr0000gp/T/ipykernel_89637/2416920381.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [27]:
documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)

/var/folders/pg/wwfxmq4x1vx_t00m5pz1z4nr0000gp/T/ipykernel_89637/1217438980.py:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [28]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [29]:
query = "How are python files parsed?"

In [30]:
raw_query_embedding = get_huggingface_embeddings(query)

raw_query_embedding

array([ 5.29357567e-02, -6.24647066e-02, -2.87437793e-02,  1.83179360e-02,
       -4.33846377e-04,  4.03238945e-02, -7.76650757e-03, -2.74391146e-03,
        2.53445171e-02, -8.10819641e-02, -8.44585430e-03, -6.59265695e-03,
        4.16187905e-02,  3.98627371e-02,  2.82911919e-02,  2.84344591e-02,
        2.65302937e-02, -2.60126553e-02,  4.16299328e-02,  3.92820574e-02,
       -5.15580177e-02,  5.83349727e-02,  5.88832889e-03,  3.46064903e-02,
       -2.46877363e-03,  2.72808708e-02,  1.07212616e-02,  4.55761366e-02,
       -1.69188995e-02, -4.85301316e-02, -3.02425176e-02, -3.29698063e-02,
        2.46010330e-02,  3.23601812e-02,  1.16030503e-06,  9.71374661e-03,
       -3.70800272e-02,  1.84201207e-02, -1.39834406e-02,  4.25722934e-02,
        6.78141117e-02, -6.66247159e-02,  2.11651698e-02, -1.11715030e-03,
       -1.80114824e-02, -7.90140033e-02,  5.93152903e-02, -5.23733683e-02,
        5.63013516e-02,  4.31279950e-02,  7.77091598e-03, -2.30586529e-02,
       -2.94572208e-02,  

In [31]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

In [32]:
top_matches

{'matches': [{'id': '1e98ddff-6a6e-4446-b7e6-729dae80c230',
              'metadata': {'source': 'src/context/language/python-parser.ts',
                           'text': 'src/context/language/python-parser.ts\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'export class PythonParser implements '
                                   'AbstractParser {\n'
                                   '  findEnclosingContext(\n'
                                   '    file: string,\n'
                                   '    lineStart: number,\n'
                                   '    lineEnd: number\n'
                                   '  ): EnclosingContext {\n'
                                   '    // TODO: Implement this method for '
                                   'Python\n'
                                   '    return null;\n'
                          

In [33]:
# contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [34]:
# contexts

['src/context/language/python-parser.ts\nimport { AbstractParser, EnclosingContext } from "../../constants";\nexport class PythonParser implements AbstractParser {\n  findEnclosingContext(\n    file: string,\n    lineStart: number,\n    lineEnd: number\n  ): EnclosingContext {\n    // TODO: Implement this method for Python\n    return null;\n  }\n  dryRun(file: string): { valid: boolean; error: string } {\n    // TODO: Implement this method for Python\n    return { valid: false, error: "Not implemented yet" };\n  }\n}\n',
 'src/context/language/javascript-parser.ts\nimport { AbstractParser, EnclosingContext } from "../../constants";\nimport * as parser from "@babel/parser";\nimport traverse, { NodePath, Node } from "@babel/traverse";\n\nconst processNode = (\n  path: NodePath<Node>,\n  lineStart: number,\n  lineEnd: number,\n  largestSize: number,\n  largestEnclosingContext: Node | null\n) => {\n  const { start, end } = path.node.loc;\n  if (start.line <= lineStart && lineEnd <= end.li

In [35]:
# augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [36]:
# print(augmented_query)

<CONTEXT>
src/context/language/python-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
export class PythonParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    // TODO: Implement this method for Python
    return null;
  }
  dryRun(file: string): { valid: boolean; error: string } {
    // TODO: Implement this method for Python
    return { valid: false, error: "Not implemented yet" };
  }
}


-------

src/context/language/javascript-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
import * as parser from "@babel/parser";
import traverse, { NodePath, Node } from "@babel/traverse";

const processNode = (
  path: NodePath<Node>,
  lineStart: number,
  lineEnd: number,
  largestSize: number,
  largestEnclosingContext: Node | null
) => {
  const { start, end } = path.node.loc;
  if (start.line <= lineStart && lineEnd <= end.line) {
    const

In [38]:
# system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

# Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
# """

# llm_response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": augmented_query}
#     ]
# )

# response = llm_response.choices[0].message.content

In [39]:
# response

'Currently, Python files are not yet parsed. The `PythonParser` class in `src/context/language/python-parser.ts` implements the `AbstractParser` interface, but both of its methods, `findEnclosingContext` and `dryRun`, are not implemented. The `findEnclosingContext` method is meant to find and return the enclosing context within the Python code, but it currently returns `null`. Similarly, the `dryRun` method, which should validate Python files, returns an object indicating it is not implemented yet (with `valid: false` and `error: "Not implemented yet"`).\n\nThus, as of the provided context, Python files do not have a parsing implementation in the codebase.'

In [40]:
# def perform_rag(query):
#     raw_query_embedding = get_huggingface_embeddings(query)

#     top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

#     # Get the list of retrieved texts
#     contexts = [item['metadata']['text'] for item in top_matches['matches']]

#     augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

#     # Modify the prompt below as need to improve the response quality
#     system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

#     Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
#     """

#     llm_response = client.chat.completions.create(
#         model="gpt-4o",
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": augmented_query}
#         ]
#     )

#     return llm_response.choices[0].message.content

In [41]:
# response = perform_rag("How is the javascript parser used?")

# print(response)

The JavaScript parser in the codebase is implemented by the `JavascriptParser` class in `src/context/language/javascript-parser.ts`. Here's a brief overview of how it is used across the codebase based on the provided context:

1. **Parser Initialization:**
   - The `JavascriptParser` is constructed and associated with specific file extensions through the `EXTENSIONS_TO_PARSERS` map in `src/constants.ts`. This map associates file extensions like `ts`, `tsx`, `js`, and `jsx` with the `JavascriptParser`.

2. **Parser Selection:**
   - The function `getParserForExtension` in `src/constants.ts` retrieves the appropriate parser for a file based on its extension by consulting the `EXTENSIONS_TO_PARSERS` map.

3. **Usage in `smarterContextPatchStrategy`:**
   - In `src/context/review.ts`, the `smarterContextPatchStrategy` function makes use of the parser by first obtaining it with `getParserForExtension(file.filename)`. If a parser is available for the file's extension (e.g., JavaScript or Typ